# Advanced Monte Carlo settings - kernels and steps

**smol** allows a lot of flexibility to set and control the specifics of montecarlo simulations, these are implemented in three types of helper classes that can be set when creating a sampler:

- [`MCKernels`](https://cedergrouphub.github.io/smol/api_reference/moca/kernel.kernels.html) define the Monte Carlo sampling algorithm
- [`MCUshers`](https://cedergrouphub.github.io/smol/api_reference/moca/kernel.mcusher.html) specify the type of step or transition.

In [1]:
import numpy as np
import json
from smol.io import load_work
from smol.moca import Ensemble, Sampler, available_mckernels, available_step_types, available_bias_types

### 0) Load the previous LNO CE with electrostatics

In [2]:
work = load_work('./data/basic_ce_ewald.mson')
expansion = work['ClusterExpansion']

### 1) Create a semigrand ensemble
The `Ensemble` class can also be used to run semigrand canonical MC by fixing relative chemical potentials.

In [3]:
from smol.moca import Ensemble

sc_matrix = np.array([
    [6, 1, 1],
    [1, 2, 1],
    [1, 1, 2]
])

chemical_potentials = {'Li+': 0, 'Vacancy': 0, 'Ni3+': 0, 'Ni4+': 0}

ensemble = Ensemble.from_cluster_expansion(
    expansion, sc_matrix, chemical_potentials=chemical_potentials
)


### 2) List the available classes

In [4]:
print(f"The available Monte Carlo kernels are:\n{available_mckernels()}\n")
print(f"The available Monte Carlo step types are:\n{available_step_types()}\n")

The available Monte Carlo kernels are:
('metropolis', 'uniformly-random', 'wang-landau', 'multicell-metropolis')

The available Monte Carlo step types are:
('flip', 'swap', 'multi-step', 'composite', 'table-flip')



### 3) Choosing the kernel and step type

The specific choices of each of the above can be done when initializing a Sampler.

Specific options for the kernel and the step type are passed as additional keyword arguments.

In [16]:
from smol.moca.kernel.mcusher import Flip

flip = Flip(ensemble.processor.get_sublattices())

sampler = Sampler.from_ensemble(
    ensemble,
    # kernel settings
    kernel_type="metropolis",  # this is the default value
    temperature=1000,
    # step type settings
    step_type="multi-step", #  make sure the step type is valid if giving chemical potentials
    mcusher=flip,  # look at the documentation of Multistep to see the options
    step_lengths=5,  # each step will be composed of 5 random flips
)

In [26]:
print(sampler.samples.metadata)

Metadata(chemical_potentials={Species Li+: 0, Vacancy vacA0+: 0, Species Ni3+: 0, Species Ni4+: 0}, cls_name='SampleContainer', kernels=[Metadata(seed=274324443849665987189735601095705236747, step=Metadata(sublattices=[(Species Li+, Vacancy vacA0+), (Species Ni3+, Species Ni4+), (Species O2-,)], sublattice_probabilities=array([0.5, 0.5]), cls_name='MultiStep', step=Metadata(sublattices=[(Species Li+, Vacancy vacA0+), (Species Ni3+, Species Ni4+), (Species O2-,)], sublattice_probabilities=array([0.5, 0.5]), cls_name='Flip'), step_lengths=array([5]), step_probabilities=array([1.])), cls_name='Metropolis')])
